# Aprendizem Profunda - Ensemble (bert -> lstm -> cnn)
### Tarefa III
1. Rúben Gonçalo Araújo da Silva pg57900   
2. José Luis Fraga Costa pg55970
3. Pedro Miguel Costa Azevedo pg57897
4. Rui Pedro Fernandes Madeira Pinto pg56010

# Implementação

### imports

1. pandas
2. tensorflow
3. sklearn

In [3]:
# Standard Library
import os
import csv

# Data Handling
import pandas as pd
import numpy as np

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression

# TensorFlow / Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Embedding, Flatten, Dense, Dropout, SimpleRNN, GRU, LSTM, 
    Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D, 
    BatchNormalization, Conv1D,MaxPooling1D
)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam, AdamW
# Hugging Face Transformers
from transformers import (
    AutoTokenizer, 
    TFAutoModelForSequenceClassification, 
    pipeline, BertTokenizer, TFBertForSequenceClassification
)


### Input dos dados

In [4]:
df = pd.read_csv("data/fixed_dataset_final.csv")
df = df.dropna(subset=["Label"])
df["Text"] = df["Text"].fillna("")
all_ids = df["ID"].fillna("")
test_ids = df["ID"].fillna("")


### Função pra gravar em csv

In [5]:
def to_csv(results_df, name):
    os.makedirs("score", exist_ok=True)
    path = f"score/{name}.csv"
    filtered_df = results_df[results_df["ID"].notna() & (results_df["ID"] != "")].copy()
    filtered_df['prefix'] = filtered_df['ID'].str.extract(r'(D[0-9])')
    filtered_df['number'] = filtered_df['ID'].str.extract(r'-(\d+)').astype(int)
    filtered_df = filtered_df.sort_values(by=['prefix', 'number'])

    with open(path, 'w') as f:
        f.write("ID\tLabel\n") 
        for index, row in filtered_df.iterrows():
            f.write(f"{row['ID']}\t{row['Label']}\n") 

In [6]:
def predict_and_save(model, X_data, ids, model_name):
    # Generate predictions
    y_pred = (model.predict(X_data) > 0.5).astype(int).flatten()

    # Convert numeric predictions to labels
    labels = ["Human" if label == 0 else "AI" for label in y_pred]
    # Create results DataFrame
    results_df = pd.DataFrame({
        "ID": ids,
        "Label": labels
    })
    to_csv(results_df, model_name)

### Train Test Split (divisão de dados)

In [7]:
# Remove duplicates by keeping the first occurrence
df_unique = df.drop_duplicates(subset=["Text"], keep="first")

# Check how many rows remain
print(f"Original dataset size: {len(df)}, Unique texts: {len(df_unique)}")

# Split the deduplicated dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df_unique["Text"], df_unique["Label"], test_size=0.2, random_state=42, stratify=df_unique["Label"]
)
test_indices = test_texts.index

train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, test_size=0.2, random_state=42, stratify=train_labels
)

# Verify no overlap
print(f"Overlap between train and test after deduplication: {len(set(train_texts) & set(test_texts))}")


Original dataset size: 4053, Unique texts: 4051
Overlap between train and test after deduplication: 0


### Tokenizer

In [8]:
# Tokenize the data
tokenizer = Tokenizer(num_words=25000)
tokenizer.fit_on_texts(train_texts)

X_train_seq = tokenizer.texts_to_sequences(train_texts)
X_val_seq = tokenizer.texts_to_sequences(val_texts)
X_test_seq = tokenizer.texts_to_sequences(test_texts)

X_train = pad_sequences(X_train_seq, maxlen=200)
X_val = pad_sequences(X_val_seq, maxlen=200)
X_test = pad_sequences(X_test_seq, maxlen=200)

# Convert string labels to numeric values
label_map = {"Human": 0, "AI": 1}  # Define mapping
y_train = np.array([label_map[label] for label in train_labels], dtype=np.float32)
y_val = np.array([label_map[label] for label in val_labels], dtype=np.float32)
y_test = np.array([label_map[label] for label in test_labels], dtype=np.float32)

## Dataset de Avaliação

In [9]:
data_set_professor = pd.read_csv("data/dataset1_inputs.csv", sep='\t', on_bad_lines='skip')
data_set_professor["Text"] = data_set_professor["Text"].fillna("")
X_professor_seq = tokenizer.texts_to_sequences(data_set_professor["Text"])
X_professor = pad_sequences(X_professor_seq, maxlen=200)  

### Early Stopping

In [10]:
early_stopping = EarlyStopping(
    monitor='val_loss',     
    patience=10,            
    restore_best_weights=True,           
)

### Reduce_lr

In [11]:
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5, 
    patience=3,
    min_lr=0.00005
)

## Bert_lstm_cnn

In [ ]:
def create_bert_model():
    model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

bert_model = create_bert_model()
bert_model.fit(X_train, np.array(y_train), epochs=5, batch_size=2)

def create_lstm_model():
    model_lstm = Sequential([
    Embedding(input_dim=25000, output_dim=32),  

    LSTM(16, activation="tanh", return_sequences=True, 
         kernel_regularizer=l2(0.005), recurrent_dropout=0.5), 
    Dropout(0.5),

    LSTM(8, kernel_regularizer=l2(0.005), recurrent_dropout=0.5, return_sequences=False),
    Dropout(0.5),

    BatchNormalization(),

    Dense(16, activation="relu", kernel_regularizer=l2(0.003)), 
    Dropout(0.7),  

    Dense(1, activation="sigmoid")
])

    optimizer = Adam(learning_rate=0.0001, clipnorm=1.0)
    model_lstm.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
    return model_lstm

lstm_model = create_lstm_model()
lstm_model.fit(X_train, np.array(y_train), epochs=3, batch_size=2)

def create_cnn_model():
    model_cnn = Sequential([
    Embedding(input_dim=25000, output_dim=32, input_length=200),

    tf.keras.layers.GaussianNoise(0.05),
    
    Conv1D(filters=32, kernel_size=5, activation=tf.keras.layers.LeakyReLU(alpha=0.1), 
           kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),
    
    GlobalMaxPooling1D(),
    Dropout(0.5),
    
    Dense(16, activation=tf.keras.layers.LeakyReLU(alpha=0.1), kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    
    Dense(1, activation='sigmoid')
])

    # Compile with AdamW and gradient clipping
    initial_learning_rate = 0.0008
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate, decay_steps=5 * len(X_train) // 128, decay_rate=0.9, staircase=True
    )
    model_cnn.compile(optimizer=AdamW(learning_rate=lr_schedule, weight_decay=0.01, clipnorm=1.0),
                      loss='binary_crossentropy',
                      metrics=['accuracy'])
    return model_cnn

cnn_model = create_cnn_model()
cnn_model.fit(X_train, np.array(y_train), epochs=3, batch_size=2)


bert_preds_train = bert_model.predict(X_train)[0][:, 1]
lstm_preds_train = lstm_model.predict(X_train)
cnn_preds_train = cnn_model.predict(X_train)

X_meta_train = np.column_stack((bert_preds_train, lstm_preds_train, cnn_preds_train))

meta_model = LogisticRegression()
meta_model.fit(X_meta_train, y_train)



All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Epoch 1/3


  56/1296 [>.............................] - ETA: 49:33 - loss: 1.1721 - accuracy: 0.4554

KeyboardInterrupt: 

# Correr Modelos

**Test Set**

In [ ]:
test_ids = df.loc[test_indices, "ID"] 


bert_preds_test = bert_model.predict(X_test)[0][:, 1]
lstm_preds_test = lstm_model.predict(X_test)
cnn_preds_test = cnn_model.predict(X_test)

X_meta_test = np.column_stack((bert_preds_test, lstm_preds_test, cnn_preds_test))
predict_and_save(meta_model, X_test, test_ids, "ensemble_dataset1")

**Avaliação**

In [ ]:

bert_preds_test = bert_model.predict(X_professor)[0][:, 1]
lstm_preds_test = lstm_model.predict(X_professor)
cnn_preds_test = cnn_model.predict(X_professor)

X_meta_test = np.column_stack((bert_preds_test, lstm_preds_test, cnn_preds_test))

predict_and_save(meta_model, X_professor, data_set_professor["ID"], "ensemble_dataset1")

In [ ]:
# 1. Carregar os arquivos
preds_df = pd.read_csv("score/ensemble_dataset1.csv", sep='\t', on_bad_lines='skip')
labels_df = pd.read_csv("data/dataset3_disclosed.csv", sep='\t', on_bad_lines='skip')

# 2. Garantir que as colunas corretas estão sendo usadas
# Assumindo que a coluna de previsões se chama 'Pred' e a de verdade se chama 'Label'
y_pred = preds_df['Label']
y_true = labels_df['Label']

# 3. Comparar previsões com os rótulos reais
print("Acurácia:", accuracy_score(y_true, y_pred))
print("Relatório de Classificação:")
print(classification_report(y_true, y_pred))

## Classification Reports

In [ ]:
print(classification_report(y_test, meta_model.predict(X_test) > 0.5))